# Detectron2 Faster R-CNN starter

This is very basic starter notebook for [TensorFlow - Help Protect the Great Barrier Reef](https://www.kaggle.com/c/tensorflow-great-barrier-reef/overview) competetion.

It uses Detectron2's Faster R-CNN pretrained network with default settings.

Changelog:

* 02-Dec: initial version, box score threshold = 0.1 (LB 0.158)
* 04-Dec: switch to Faster R-CNN X101 FPN (LB 0.269)

Hope it will be useful, enjoy)

In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

In [ ]:
import sys
import os
import random
import logging

from ast import literal_eval

import numpy as np
import pandas as pd

import cv2

import matplotlib.pyplot as plt

import torch, torchvision

import greatbarrierreef

In [ ]:
from detectron2 import model_zoo
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

In [ ]:
# configure logging to see info messages from detectron
root = logging.getLogger()
root.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(levelname)s: %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [ ]:
# data directory
DATA_DIR = '../input/tensorflow-great-barrier-reef'

# function to return competition training data in Detectron format
def get_tf_gbreef_train():
    df_train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
                           
    items = []
    for index, row in df_train.iterrows():
        # https://www.kaggle.com/c/tensorflow-great-barrier-reef/data 
        # train/ - Folder containing training set photos of the form video_{video_id}/{video_frame_number}.jpg
        video_fn = os.path.join(DATA_DIR, 'train_images', f'video_{row["video_id"]}', f'{row["video_frame"]}.jpg')
        boxes = []
        for b in literal_eval(row['annotations']):
            boxes.append({'bbox': [b['x'], b['y'], b['width'], b['height']],
                          'bbox_mode': BoxMode.XYWH_ABS,
                          'category_id': 0})
        items.append({'file_name': video_fn,
                      'height': 720,
                      'width': 1280, 
                      'image_id': row['image_id'],
                      'annotations': boxes})
                     
    return items

# register train dataset
DatasetCatalog.register('tf_gbreef_train', get_tf_gbreef_train)
ds_train = DatasetCatalog.get('tf_gbreef_train')

In [ ]:
# show some random images and bounding boxes from training set
nrows, ncols = 5, 2
fig, ax = plt.subplots(nrows, ncols, figsize=(20,31))

displayed = 0
for ti in random.sample(range(23500), 10000):
    d = ds_train[ti]
    if 0 == len(d['annotations']): continue  # skip images without bounding boxes
        
    img = cv2.cvtColor(cv2.imread(d["file_name"]), cv2.COLOR_BGR2RGB)
    for a in d['annotations']:
        b = a['bbox']
        cv2.rectangle(img, (b[0], b[1]), (b[0]+b[2], b[1]+b[3]), color=(255, 255, 255), thickness=2)
    
    ax[displayed // ncols, displayed % ncols].grid(False)
    ax[displayed // ncols, displayed % ncols].axis('off')
    ax[displayed // ncols, displayed % ncols].imshow(img)
    
    displayed += 1
    if nrows * ncols <= displayed: break
        
plt.tight_layout()
plt.show()

In [ ]:
# get config and merge it with Faster R-CNN config from Detectron zoo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))

cfg.DATASETS.TRAIN = ('tf_gbreef_train',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = "../input/detectron2-faster-rcnn-x101/model_final_68b088.pkl"  # get weigths from attached dataset
cfg.SOLVER.IMS_PER_BATCH = 2  # batch size
cfg.SOLVER.BASE_LR = 0.001  # learning rate
cfg.SOLVER.MAX_ITER = 20000  # dataset has ~5000 images with annotations so 100000 iterations is ~40 epochs with batch size of 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # we have single class in this competition

In [ ]:
# make output dir, created default trainer and go ahead)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# pickup weights from training and create predictor
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set threshold for boxes to return
predictor = DefaultPredictor(cfg)

In [ ]:
# simple function to format predictions
def format_predictions(out):
    l = []
    for i in range(len(out['instances'])):
        box = out['instances'].pred_boxes[i].tensor.cpu().numpy().flatten()
        score = out['instances'].scores[0].cpu().numpy()
        # XYXY to XYWH
        l.append(f'{score:.3f} {box[0]:.0f} {box[1]:.0f} {(box[2]-box[0]):.0f} {(box[3]-box[1]):.0f}')
    return ' '.join(l)

# create prediction env and iterator
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

# iterate over test images and do prediction
for (pixel_array, sample_prediction_df) in iter_test:
    out = predictor(pixel_array)
    sample_prediction_df['annotations'] = format_predictions(out)
    env.predict(sample_prediction_df)